<a href="https://colab.research.google.com/hardayaleva5/EVA5_Phase1_Assignment/tree/master/Session_2/blob/master/EVA5_Session2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import pytorch packages
from __future__ import print_function
# import pytorch main package
import torch
# import Base class for all neural network modules.
import torch.nn as nn
import torch.nn.functional as F
# import torch.optim is a package implementing various optimization algorithms.
import torch.optim as optim
# import torchvision packge & dataset.
# All datasets are subclasses of torch.utils.data.Dataset 
from torchvision import datasets, transforms


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Help Conv2d syntex with default values
        # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        # Before convolution RF:1x1
        #input: 28x28x1  output: 28x28x1 RF:3x3 (3x3 increase RF by 2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1) 
        
        #input: 28x28x32  output: 28x28x64 RF:5x5 (3x3 increase RF by 2)
        # No of parameter: 3*3*32(in_channels)*64(out_channels) = 18432 + bias(64) = 18496
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) 
        
        #input: 14x14x64  output: 14x14x64 RF:10x10 (2x2 MP multiply RF by 2 or double the existing RF)
        self.pool1 = nn.MaxPool2d(2, 2)  

        #input: 14x14x64  output: 14x14x128 RF:12x12 (3x3 increase RF by 2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1) 
        
        #input: 14x14x128  outtput: 14x14x256 RF:14x14 (3x3 increase RF by 2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) 
        
        #input: 14x14x256  output:7x7x256 RF:28x28 (2x2 MP multiply RF by 2 or double the existing RF)
        self.pool2 = nn.MaxPool2d(2, 2) 
        
        #input:7x7x256  output: 5x5x512 RF:30x30  (3x3 increase RF by 2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3) 
        
        #input:5x5x512  output:3x3x1024 RF:32x32 (3x3 increase RF by 2)
        self.conv6 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3) 
        
        #input:3x3x1024   output: 1x1x10 
        self.conv7 = nn.Conv2d(in_channels=1024, out_channels=10, kernel_size=3) 

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) #performs conv1 -> relu -> conv2 -> relu -> pool1
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) #performs conv3 -> relu -> conv4 -> relu -> pool2 
        x = F.relu(self.conv6(F.relu(self.conv5(x))))             #performs conv5 -> relu -> conv6 -> relu
        x = F.relu(self.conv7(x))                                 #performs conv7 -> relu
        print(x.view(-1, 10))
        
        x = x.view(-1, 10)                                      # view perform reshape operation. Here convert x as 1x10 matrics
                                                        # Applies the \log(\text{Softmax}(x))log(Softmax(x)) function to an n-dimensional input Tensor
        return F.log_softmax(x)

In [5]:
!pip install torchsummary
from torchsummary import summary

# check if nvidia cuda gpu is available use it otherwise shift to cpu
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

#transfer the model to the above selected device
model = Net().to(device)

# print the summary of the model
summary(model, input_size=(1, 28, 28))

tensor([[0.0135, 0.0000, 0.0026, 0.0000, 0.0000, 0.0019, 0.0000, 0.0140, 0.0041,
         0.0042],
        [0.0137, 0.0000, 0.0032, 0.0000, 0.0000, 0.0023, 0.0000, 0.0145, 0.0032,
         0.0046]], device='cuda:0', grad_fn=<ViewBackward>)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [6]:
#Seed the model to obtain consistent results
torch.manual_seed(1)
batch_size = 128 #Set the batch size as 128  currently (it can be any value in terms of 2^x)

#This makes sure that the data stays in the memory
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


# load the train data and perform standard normalization
# Normalize does the below operation for each channel-
# image = (image - mean) / std
# The parameters mean, std
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# load the test data and perform standard normalization
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    
    #traning model
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        
        #moves data to device
        data, target = data.to(device), target.to(device)
        
        #sets gradients to zero
        optimizer.zero_grad()
        
        #get the model output for the data
        output = model(data)
        
        #loss functio will be negative log likeli-hood
        loss = F.nll_loss(output, target)
         
         #backpropagation
        loss.backward()
        #updates the parameters based on the current gradient
        optimizer.step()
        
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    
    #sets the model in eval mode
    model.eval()

    #set test loss to zero
    test_loss = 0

     # number of correct classifications
    correct = 0
    #set gradients to zero as it is in test mode we dont require gradients
    with torch.no_grad():
        for data, target in test_loader:
            #sends data to device
            data, target = data.to(device), target.to(device)

             #get the output
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
#move the model to device
model = Net().to(device)

#set ptimizer as Stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=2.302138566970825 batch_id=5:   1%|          | 5/469 [00:00<01:29,  5.19it/s]

tensor([[0.0000, 0.0000, 0.0066,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0088,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0066,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0083,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0100,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0074,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.0000, 0.0000, 0.0075,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0090,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0054,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0053,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0034,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0060,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.0000e+00, 0.0000e+00, 6.0568e-05,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.

loss=2.3005120754241943 batch_id=13:   3%|▎         | 13/469 [00:00<00:49,  9.20it/s]

tensor([[0.0000, 0.0000, 0.0044,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0083,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0084,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0075,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0121,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0022,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.0000, 0.0000, 0.0136,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0151,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0141,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0203,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0053,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0060,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0.0000, 0.0000, 0.0113,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0137,  ..., 0.000

loss=2.2980566024780273 batch_id=21:   4%|▍         | 21/469 [00:00<00:29, 14.94it/s]

...

loss=1.801456093788147 batch_id=464:  99%|█████████▉| 465/469 [00:13<00:00, 34.68it/s]

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 6.8200,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.5007,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ViewBackward>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]

loss=1.795615315437317 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.17it/s]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0218, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0')
tensor([[0., 0., 0.,  ..., 0., 0., 0